In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('./..'))))

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam

from Data.Dataset import FinTextDataset
from Data.DataLoader import DataLoader
from Model.MainModel import FinTextModel
from train import TrainingApp

2023-01-18 17:30:00.564314: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 17:30:02.021304: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-18 17:30:02.021600: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-18 17:30:02.021611: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
text1 = "1900년 8월 8일, 파리에서 개최된 국제수학학회에서 힐베르트는 역사에 길이 남을 강연을 했다. 그는 자신이 가장 중요하다고 여긴 23개의 미해결 문제를 사람들 앞에 제시하였는데, 이들 중 일부는 수학의 일반 분야에 속하는 문제였고 나머지 대부분은 논리적 기초와 관련된 것들이었다. 힐베르트가 이런 문제들을 제기한 것은 수학계의 관심을 끌어 자신의 연구 계획을 분담, 수행할 학자들을 모집하기 위한 포석이었다. 그는 모순 없는 수학체계를 확립하는 대계획에 전세계의 수학자들을 골고루 참여시키고 싶었던 것이다. 힐베르트의 야심찬 의도는 그의 묘비에 잘 표현되어 있다."
text2 = '태어나서 처음 들은 한국어 문장, 또는 영어를 배울 때 처음 배운 문장이 무엇이었는지를 기억하는 일은 불가능하다. 하지만 몇몇 언어의 경우에는 기억한다. 처음으로 들은 중국어 문장은 "我爱你"(wǒ ài nǐ, 나는 너를 사랑한다)였고, 초등학교 4학년 무렵 사촌 누나가 알려준 것이었다. 라틴어 문장 중 내가 기억하기에 가장 처음 알게 된 것은 카이사르의 "왔노라, 보았노라, 이겼노라"였다. 데카르트의 "나는 생각한다, 고로 존재한다"이 그 다음일 것이다. 두 가지 모두 거의 같은 시기에 이원복 선생님의 《먼나라 이웃나라》 시리즈를 읽고 알게 되었을 가능성이 크다. 그 중 데카르트의 문장은 리처드 파인만의 글 중 《남이야 뭐라 하건!》에 수록된 같은 제목의 글에도 언급되어 있어서, 고등학생 시절 그 책을 읽으며 다시 접하게 되었다.'
text3 = "인용한 것과 같이 사이먼 싱의 《페르마의 마지막 정리》에도 그 문장이 나온다. 이 문장을 그 책에서 처음 접한 것은 분명하다. 나는 사이먼 싱의 책을 초등학교 6학년일 때 읽었고, 그 내용을 거의 기억할 정도로 몰입해서 읽었다. 그 이전에는 수학 대중서를 읽은 적이 없었다. 하지만 이 문장을 그 책에서 읽은 것으로 기억하지는 못한다. 당시에는 독일어라는 언어에 대해 크게 의식하고 있지 않았다. 영어의 조동사 'must'나 'will'조차 제대로 알고 있었을까 싶다. (초등학생 무렵의 나는 사람이 추구해야 할 진정한 지식은 수학이나 과학이라고 생각했다. 일찍부터 영어를 배우려고 하는 주변의 동급생들을 한심하다고 생각했다. 나는 지금도 초등학생에게 영어를 가르치는 일에 대해서는 굉장히 회의적이다.)"
text4 = "그 이후로도 나는 수학이나 물리학에 대한 대중적인 책과 꽤 전문적인 책들을 많이 읽었다. 결국은 어떤 책에서 위의 문장을 다시 읽게 되었다. 너무나 유명한 문장이니, 사실은 어떤 책에서 읽었어도 이상하지 않을 것이다. 아마 중학교 3학년이나 고등학교 1학년 무렵이었던 것 같다. 고등학생 때 읽은 존 배로의 《수학, 천상의 학문》, 제3장, 세번째 절 '힐베르트의 계획'에도 이 문장이 인용되는데, 정확하게 인용된 것은 아니다."

In [3]:
df = pd.DataFrame(
    {
        'ArticleText': [[text1, text2], [text3, text4]],
        'CommunityText': [[text1, text2], [text3, text4]],
        'MetricIndex': [0.1, 0.2],
        'Open': [1., 1.],
        'High': [2., 2.],
        'Low': [0., 0.],
        'Close': [.5, .5],
        'Label': [1, 0]
    }
)

In [4]:
df.loc[1]

ArticleText      [인용한 것과 같이 사이먼 싱의 《페르마의 마지막 정리》에도 그 문장이 나온다. 이...
CommunityText    [인용한 것과 같이 사이먼 싱의 《페르마의 마지막 정리》에도 그 문장이 나온다. 이...
MetricIndex                                                    0.2
Open                                                           1.0
High                                                           2.0
Low                                                            0.0
Close                                                          0.5
Label                                                            0
Name: 1, dtype: object

In [5]:
dataset = FinTextDataset(df)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 

In [7]:
dataset.feature_df

,article_matrix,community_matrix,community_metric_index,price_vector
0,"[[[tensor(0.0672), tensor(-0.6846), tensor(-0....","[[[tensor(0.1826), tensor(0.9824), tensor(0.20...","[tensor(0.1000), tensor(0.2000)]","[[tensor(1.), tensor(2.), tensor(0.), tensor(0..."
